In [1]:
import pandas as pd
import polars as pl

In [2]:
input_files = '.\\input_files\\'
file = "4-2121377_Shriners Children's Boston_standardcharges.csv"

df = pd.read_csv(input_files + file, dtype=str)

In [3]:
df.columns = df.columns.str.strip()
df.rename(columns={
    'Code': 'code',
    'Description': 'description',
    'Type': 'setting',
}, inplace=True)

In [4]:
df.drop(columns=['Package/Line_Level', 'Derived contracted rate'], inplace=True)

In [5]:
cols = df.columns.tolist()
id_vars = cols[:3]
value_vars = cols[3:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [6]:
df['standard_charge'] = df['standard_charge'].str.strip()
df.loc[df['standard_charge'] == 'N/A', 'standard_charge'] = pd.NA
df.dropna(subset=['standard_charge'], inplace=True)

df['setting'] = df['setting'].str.lower()



In [7]:
df.loc[df['code'].str.len() == 1, 'ms_drg'] = df['code'].str.zfill(3)
df.loc[df['code'].str.len() == 3, 'ms_drg'] = df['code']
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']

In [8]:
mapping = {
    'Gross Charge': 'gross',
    'Discounted cash price': 'cash',
    'De-identified min contracted rate': 'min',
    'De-identified max contracted rate': 'max',
}
df['payer_category'] = df['payer'].map(mapping).fillna('payer')

In [9]:
file_locations = {
    'Boston': '223304',
    'Greenville': '423300',
    'Hawaii': '123301',
    'Northern': '053311',
    'Ohio': '363308',
    'Philadelphia': '393309',
    'Spokane': '503302',
    'Chicago': '143302',
    'Louis': '263304',
    'Portland': '383300',
    'Texas': '453311'
}

for location, ccn in file_locations.items():
    if location in file:
       id = ccn

df['hospital_id'] = ccn

In [17]:
df['standard_charge'] = df['standard_charge'].str.replace(',', '', regex=False)

In [19]:
df['standard_charge'] = pd.to_numeric(df['standard_charge'])

In [23]:
df['ms_drg'].fillna('', inplace=True)
df['hcpcs_cpt'].fillna('', inplace=True)
df['code'].fillna('', inplace=True)

,code,description,setting,payer,standard_charge,ms_drg,hcpcs_cpt,payer_category,hospital_id


In [28]:
df.sample(10)

,code,description,setting,payer,standard_charge,ms_drg,hcpcs_cpt,payer_category,hospital_id
31457,70546,MR ANGIOGRAPHY HEAD WITH & WITHOUT CONTRAST,outpatient,BCBS of MA-Commercial other_Avg,52.89,,70546,payer,453311
11455,3,"ECMO OR TRACH WITH MV >96 HRS OR PDX EXC FACE,...",inpatient,Aetna-Commercial PPO/Open Access_Avg,0.00,003,,payer,453311
9227,80061,"BLOOD TEST, LIPIDS (CHOLESTEROL AND TRIGLYCERI...",outpatient,Aetna-Commercial HMO/POS_Avg,0.00,,80061,payer,453311
13560,92620,SLP CENTRAL AUDITORY FCN INIT 60 MIN UNIT,outpatient,Aetna-Medicare Advantage HMO_Avg,260.40,,92620,payer,453311
126274,26989,OTHER PROCEDURE ON HANDS OR FINGERS,outpatient,United Healthcare -Commercial HMO/POS_Avg,0.00,,26989,payer,453311
13643,96001,MAL 3-D KINEMATICS/PLANTAR PRESSURE,outpatient,Aetna-Medicare Advantage HMO_Avg,261.73,,96001,payer,453311
10837,82308,CALCITONIN LEVEL,outpatient,Aetna-Commercial PPO/Open Access_Avg,886.43,,82308,payer,453311
12725,90837,"PSYCHOTHERAPY, 1 HOUR",outpatient,Aetna Better Health -Medicaid HMO_Avg,0.00,,90837,payer,453311
654,87073,"CULTURE, BACTERIAL; QUANTITATIVE, ANAEROBIC W/...",outpatient,Gross Charge,84.90,,87073,gross,453311
13029,73701,CT ANKLE BILATERAL WITH CONTRAST,outpatient,Aetna-Medicare Advantage HMO_Avg,71.72,,73701,payer,453311
